<a href="https://colab.research.google.com/github/ParkWonjeong/Limitless/blob/main/limitless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 금융 데이터 수집 및 차트 작성을 위한 라이브러리 설치
!pip install yfinance pandas_ta plotly

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio
# Colab 전용 렌더러로 설정 (차트가 안 보일 때 해결법)
pio.renderers.default = 'colab'

In [ ]:
# 비트코인(BTC-USD) 데이터 가져오기 (1시간 봉 기준)
df = yf.download("BTC-USD", period="1mo", interval="1h")

# 데이터 확인
print(df.tail())

/tmp/ipython-input-3682059198.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Price                             Close          High           Low  \
Ticker                          BTC-USD       BTC-USD       BTC-USD   
Datetime                                                              
2026-01-29 09:00:00+00:00  87768.679688  87995.156250  87768.679688   
2026-01-29 10:00:00+00:00  87803.015625  87847.554688  87709.171875   
2026-01-29 11:00:00+00:00  87690.210938  87909.734375  87629.210938   
2026-01-29 12:00:00+00:00  87951.867188  87951.867188  87709.984375   
2026-01-29 13:00:00+00:00  87845.445312  87996.101562  87845.445312   

Price                              Open      Volume  
Ticker                          BTC-USD     BTC-USD  
Datetime                                             
2026-01-29 09:00:00+00:00  87929.890625           0  
2026-01-29 10:00:00+00:00  87763.289062           0  
2026-01-29 11:00:00+00:00  87807.242188           0  
2026-01-29 12:00:00+00:00  87709.984375   689393664  
2026-01-29 13:00:00+00:00  87966.601562  1240965120  

In [ ]:
# Colab 출력 강제 설정
pio.renderers.default = 'colab'

# 데이터 다시 불러오기 및 정리
df = yf.download("BTC-USD", period="7d", interval="1h")
df.columns = df.columns.droplevel('Ticker') # 다중 인덱스 해결

# 차트 생성
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close']
)])

fig.update_layout(
    title='비트코인(BTC-USD) 1시간 봉 차트',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

/tmp/ipython-input-4036211935.py:5: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


## 이동평균선 추가

In [ ]:
import pandas_ta as ta # 지표 계산을 위한 라이브러리

# 1. 이동평균선 계산
# df['Close'] 데이터가 한 줄(Series)인지 확인 후 계산
df['MA20'] = ta.sma(df['Close'], length=20)
df['MA50'] = ta.sma(df['Close'], length=50)

# 2. 기존 캔들스틱 차트에 선 추가하기
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'],
    name="Candlestick"
)])

# MA20 선 추가 (주황색)
fig.add_trace(go.Scatter(x=df.index, y=df['MA20'],
                         mode='lines', name='MA 20',
                         line=dict(color='orange', width=1.5)))

# MA50 선 추가 (하늘색)
fig.add_trace(go.Scatter(x=df.index, y=df['MA50'],
                         mode='lines', name='MA 50',
                         line=dict(color='cyan', width=1.5)))

# 3. 레이아웃 설정
fig.update_layout(
    title='BTC-USD 이동평균선 전략 차트',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

## RSI 지표 추가

In [ ]:
from plotly.subplots import make_subplots

# 1. Colab 출력 및 데이터 준비
pio.renderers.default = 'colab'
df = yf.download("BTC-USD", period="7d", interval="1h")
df.columns = df.columns.droplevel('Ticker') # 다중 인덱스 제거

# 2. 지표 계산 (MA + RSI)
df['MA20'] = ta.sma(df['Close'], length=20)
df['MA50'] = ta.sma(df['Close'], length=50)
df['RSI'] = ta.rsi(df['Close'], length=14) # 기본값인 14일 기준

# 3. 차트 레이아웃 설정 (2층 차트 만들기)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=('BTC 가격 및 이동평균선', 'RSI 지표'),
                    row_heights=[0.7, 0.3]) # 7:3 비율

# 4. 상단(1행): 캔들스틱 및 이동평균선 추가
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'], name="BTC"), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], name='MA20', line=dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], name='MA50', line=dict(color='cyan')), row=1, col=1)

# 5. 하단(2행): RSI 추가
fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)

# RSI 기준선(30, 70) 추가
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

# 6. 마무리 설정
fig.update_layout(
    height=800,
    title='Clowder 봇: 기술적 분석 대시보드',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

/tmp/ipython-input-1928676559.py:5: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio

# 1. 환경 설정 및 데이터 수집
pio.renderers.default = 'colab'
df = yf.download("BTC-USD", period="3mo", interval="1d") # 스윙 전략이므로 일봉(1d) 기준
df.columns = df.columns.droplevel('Ticker')

# 2. Smart Swing v4 기술적 지표 계산
df['MA5'] = ta.sma(df['Close'], length=5)
df['MA20'] = ta.sma(df['Close'], length=20)

# 3. 매수 신호(Golden Cross) 로직
# 5일선이 20일선을 상향 돌파할 때
df['Signal'] = (df['MA5'] > df['MA20']) & (df['MA5'].shift(1) <= df['MA20'].shift(1))

# 4. 차트 시각화
fig = go.Figure()

# 캔들스틱 추가
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'], name="BTC Price"))

# 이동평균선 추가 (5일, 20일)
fig.add_trace(go.Scatter(x=df.index, y=df['MA5'], name='MA5 (단기)', line=dict(color='yellow', width=1.5)))
fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], name='MA20 (장기)', line=dict(color='cyan', width=1.5)))

# 매수 신호 지점 표시
buy_signals = df[df['Signal'] == True]
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals['Low'] * 0.98,
    mode='markers', name='BUY Signal',
    marker=dict(symbol='triangle-up', size=12, color='lime')
))

# 5. [핵심] 리스크 관리 라인 시각화 (가장 최근 신호 기준)
if not buy_signals.empty:
    latest_buy_price = buy_signals['Close'].iloc[-1]
    tp_price = latest_buy_price * 1.15 # 익절가 +15%
    sl_price = latest_buy_price * 0.98 # 손절가 -2%

    # 익절선 표시
    fig.add_hline(y=tp_price, line_dash="dot", line_color="green",
                  annotation_text=f"Target (+15%): {tp_price:.0f}")
    # 손절선 표시
    fig.add_hline(y=sl_price, line_dash="dot", line_color="red",
                  annotation_text=f"Stop Loss (-2%): {sl_price:.0f}")

# 레이아웃 설정
fig.update_layout(
    title='Smart Swing v4 전략 대시보드 (AI + Technical Analysis)',
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    height=700
)

fig.show()

/tmp/ipython-input-2274428590.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


현재 기술적 지표만 추가한 상태에서의 백테스팅

In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# 1. 데이터 준비 (최근 1년 일봉 데이터)
df = yf.download("BTC-USD", period="1y", interval="1d")
df.columns = df.columns.droplevel('Ticker')

# 2. 지표 계산
df['MA5'] = ta.sma(df['Close'], length=5)
df['MA20'] = ta.sma(df['Close'], length=20)

# 3. 백테스팅 변수 설정
initial_balance = 10000000  # 초기 자본 1,000만원
balance = initial_balance
position = 0                # 보유 수량
buy_price = 0               # 매수 가격
history = []                # 수익률 기록

# 4. 시뮬레이션 루프
for i in range(1, len(df)):
    current_price = df['Close'].iloc[i]
    prev_ma5 = df['MA5'].iloc[i-1]
    prev_ma20 = df['MA20'].iloc[i-1]
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]

    # [매수 조건]: 골든크로스 발생 시 & 현재 무포지션일 때
    if position == 0 and curr_ma5 > curr_ma20 and prev_ma5 <= prev_ma20:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"[{df.index[i].date()}] 매수 | 가격: {buy_price:,.0f}")

    # [매도 조건]: 포지션 보유 중일 때
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # 1) 익절: +15% 이상 달성
        # 2) 손절: -2% 이하 하락
        # 3) 추세 이탈: 데드크로스 발생 (추가 필터)
        if profit_loss >= 0.15 or profit_loss <= -0.02 or (curr_ma5 < curr_ma20):
            balance = position * current_price
            history.append(profit_loss)
            status = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"[{df.index[i].date()}] 매도 ({status}) | 가격: {current_price:,.0f} | 수익률: {profit_loss*100:.2f}%")
            position = 0
            buy_price = 0

# 최종 정산
final_assets = balance if position == 0 else position * df['Close'].iloc[-1]
total_return = (final_assets - initial_balance) / initial_balance * 100
win_rate = len([x for x in history if x > 0]) / len(history) * 100 if history else 0

print("\n" + "="*30)
print(f"▶ 최종 자산: {final_assets:,.0f}원")
print(f"▶ 누적 수익률: {total_return:.2f}%")
print(f"▶ 매매 횟수: {len(history)}회")
print(f"▶ 승률: {win_rate:.2f}%")
print("="*30)

/tmp/ipython-input-4046244650.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

[2025-02-23] 매수 | 가격: 96,274
[2025-02-24] 매도 (손절) | 가격: 91,418 | 수익률: -5.04%
[2025-03-23] 매수 | 가격: 86,054
[2025-03-29] 매도 (손절) | 가격: 82,598 | 수익률: -4.02%
[2025-04-14] 매수 | 가격: 84,542
[2025-05-08] 매도 (익절) | 가격: 103,241 | 수익률: 22.12%
[2025-06-10] 매수 | 가격: 110,257
[2025-06-12] 매도 (손절) | 가격: 105,929 | 수익률: -3.93%
[2025-06-27] 매수 | 가격: 107,088
[2025-07-29] 매도 (추세이탈) | 가격: 117,922 | 수익률: 10.12%
[2025-08-10] 매수 | 가격: 119,307
[2025-08-18] 매도 (손절) | 가격: 116,252 | 수익률: -2.56%
[2025-09-10] 매수 | 가격: 113,955
[2025-09-24] 매도 (추세이탈) | 가격: 113,329 | 수익률: -0.55%
[2025-10-02] 매수 | 가격: 120,681
[2025-10-10] 매도 (손절) | 가격: 113,214 | 수익률: -6.19%
[2025-10-27] 매수 | 가격: 114,119
[2025-10-29] 매도 (손절) | 가격: 110,055 | 수익률: -3.56%
[2025-12-03] 매수 | 가격: 93,528
[2025-12-05] 매도 (손절) | 가격: 89,388 | 수익률: -4.43%
[2026-01-01] 매수 | 가격: 88,732
[2026-01-21] 매도 (추세이탈) | 가격: 89,377 | 수익률: 0.73%

▶ 최종 자산: 9,937,940원
▶ 누적 수익률: -0.62%
▶ 매매 횟수: 11회
▶ 승률: 27.27%


## 딥러닝 모델 추가

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. 데이터 수집 (최근 5년)
data = yf.download("BTC-USD", period="5y", interval="1d")
data.columns = data.columns.droplevel('Ticker')

# 2. 데이터 정규화 (0~1 사이 값으로 변환)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

# 3. 학습 데이터 생성 (과거 60일의 데이터를 보고 다음 날을 예측)
prediction_days = 60
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

/tmp/ipython-input-2643635898.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [ ]:
model = Sequential()

# 첫 번째 LSTM 레이어
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2)) # 과적합 방지

# 두 번째 LSTM 레이어
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# 출력 레이어 (내일 가격 예측)
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 0.0321
Epoch 2/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0036
Epoch 3/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0032
Epoch 4/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - loss: 0.0032
Epoch 5/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - loss: 0.0026
Epoch 6/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.0027
Epoch 7/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - loss: 0.0023
Epoch 8/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.0025
Epoch 9/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0025
Epoch 10/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0023
Epoch 11/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.0022
Epoch 12/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.0023
Epoch 13/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0018
Epoch 14/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.0019
Epoch 15/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.0022
Epoch 16/25
56/

In [ ]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # [AI 예측 구간]
    # 현재 시점까지의 데이터를 스케일러로 변환 (이미 학습 때 만든 scaled_data 활용)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    pred_scaled = model.predict(last_60_days_input, verbose=0)
    pred_price = scaler.inverse_transform(pred_scaled)[0][0]

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma5 = df['MA5'].iloc[i-1]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수 조건: 기술적 지표 + AI 필터]
    # AI 조건: 내일 예측가가 오늘 종가보다 높을 것 (상승 예측)
    is_ai_bullish = pred_price > current_price
    is_golden_cross = (curr_ma5 > curr_ma20 and prev_ma5 <= prev_ma20)

    if is_golden_cross:
      print(f"[{df.index[i].date()}] 골든크로스 발생! 하지만 AI의 판단은? -> {'상승예측' if is_ai_bullish else '하락예측'}")

    if position == 0 and is_golden_cross and is_ai_bullish:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"[{df.index[i].date()}] AI승인 매수 | 가격: {buy_price:,.0f} (예측가: {pred_price:,.0f})")

    # [매도 조건: 기존과 동일]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or (curr_ma5 < curr_ma20):
            balance = position * current_price
            history.append(profit_loss)
            position = 0
            print(f"[{df.index[i].date()}] 매도 완료 | 수익률: {profit_loss*100:.2f}%")

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-04-14] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-06-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-06-27] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-08-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-09-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-10-02] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-10-27] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-12-03] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2026-01-01] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측

최종 수익률: 0.00%


모델 변경: LSTM -> GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# [이 부분이 핵심!] 모델의 구조를 GRU로 정의합니다.
model = Sequential()

# 첫 번째 GRU 레이어 (입력 데이터의 피처가 3개라고 가정: Price, RSI, Volume)
model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

# 두 번째 GRU 레이어
model.add(GRU(units=50, return_sequences=False))
model.add(Dropout(0.2))

# 출력 레이어 (0~1 사이의 확률을 뱉도록 Sigmoid 사용)
model.add(Dense(units=1, activation='sigmoid'))

# 컴파일 (분류 문제이므로 binary_crossentropy 권장)
model.compile(optimizer='adam', loss='binary_crossentropy')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Early Stopping 설정
# monitor='val_loss': 검증 오차를 관찰
# patience=10: 오차가 개선되지 않더라도 10번은 더 지켜봄
# restore_best_weights=True: 학습 중단 후 가장 성적이 좋았던 가중치로 복구
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 2. 가장 좋은 모델을 파일로 저장하는 설정 (선택 사항)
checkpoint = ModelCheckpoint(
    'best_limitless_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# 3. 모델 학습 실행
history = model.fit(
    x_train, y_train,
    epochs=100,           # Early Stopping이 있으므로 에포크를 넉넉히 잡음
    batch_size=32,
    validation_split=0.2, # 데이터의 20%를 검증용으로 사용
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

Epoch 1/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.6398

45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 88ms/step - loss: 0.6373 - val_loss: 0.5152
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.4824

45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - loss: 0.4823 - val_loss: 0.4990
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.4758 - val_loss: 0.5018
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.4757 - val_loss: 0.5024
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - loss: 0.4747 - val_loss: 0.5027
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.4782 - val_loss: 0.5033
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - loss: 0.4727 - val_loss: 0.5081
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - loss: 0.4748 - val_loss: 0.5039
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - loss: 0.4773 - val_loss: 0.4997
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.4757

45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.4757 - val_loss: 0.4986
Epoch 11/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.4776

45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - loss: 0.4774 - val_loss: 0.4984
Epoch 12/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.4632

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4693 - val_loss: 0.4934
Epoch 16/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.4744 - val_loss: 0.4961
Epoch 17/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 0.4762 - val_loss: 0.4942
Epoch 18/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - loss: 0.4673 - val_loss: 0.4944
Epoch 19/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4751 - val_loss: 0.4969
Epoch 20/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 0.4751 - val_loss: 0.4941
Epoch 21/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - loss: 0.4701 - val_loss: 0.4945
Epoch 22/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.4691

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4692 - val_loss: 0.4925
Epoch 23/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.4773 - val_loss: 0.4936
Epoch 24/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.4691 - val_loss: 0.4932
Epoch 25/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - loss: 0.4796 - val_loss: 0.4934
Epoch 26/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 0.4708 - val_loss: 0.4929
Epoch 27/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 0.4756 - val_loss: 0.4929
Epoch 28/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - loss: 0.4764 - val_loss: 0.4958
Epoch 29/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 0.4722 - val_loss: 0.4955
Epoch 30/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.4677

45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 0.4679 - val_loss: 0.4923
Epoch 31/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 0.4723 - val_loss: 0.4927
Epoch 32/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.4707

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 0.4708 - val_loss: 0.4921
Epoch 33/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4750 - val_loss: 0.4928
Epoch 34/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - loss: 0.4721 - val_loss: 0.4945
Epoch 35/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.4755

45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - loss: 0.4754 - val_loss: 0.4917
Epoch 36/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.4783 - val_loss: 0.4921
Epoch 37/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 112ms/step - loss: 0.4672 - val_loss: 0.4920
Epoch 38/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - loss: 0.4679 - val_loss: 0.4934
Epoch 39/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - loss: 0.4733 - val_loss: 0.4932
Epoch 40/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - loss: 0.4693 - val_loss: 0.4928
Epoch 41/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - loss: 0.4750 - val_loss: 0.4926
Epoch 42/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4726 - val_loss: 0.4919
Epoch 43/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - loss: 0.4674 - val_loss: 0.4931
Epoch 44/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4704 - val_loss: 0.4957
Epoch 45/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.4758 - val_loss: 0.4928


In [ ]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측: 확률값 가져오기 (0~1 사이)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    prob = model.predict(last_60_days_input, verbose=0)[0][0] # Sigmoid 결과값

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1] # 20일선 기울기 확인용

    # [과제 기반 3중 필터 + AI 확률]
    # 1) 가격이 20일선 위 2) 20일선이 우상향 3) 5일선 > 20일선
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)

    # AI 조건: 확률이 0.465를 넘는지 확인
    is_ai_approve = prob > 0.38

    if is_golden_status:
        print(f"[{df.index[i].date()}] 골든 상태! | AI확률: {prob:.4f} | 이평기울기: {'상승' if curr_ma20 > prev_ma20 else '하락'}")

    # 최종 매수 결정
    if position == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"🚀 [{df.index[i].date()}] 진입! | 가격: {buy_price:,.0f} | 확률: {prob:.4f}")

    # [매도 조건: 과제 로직 - 익절 15%, 손절 -2%, 추세이탈 ma20]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # A. 익절 15% / B. 손절 -2% / C. 가격이 20일선 하향 돌파
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance = position * current_price
            history.append(profit_loss)
            reason = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 매도 ({reason}) | 수익률: {profit_loss*100:.2f}%")
            position = 0

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-03-30] 골든 상태! | AI확률: 0.3855 | 이평기울기: 상승
[2025-04-14] 골든 상태! | AI확률: 0.4308 | 이평기울기: 하락
[2025-04-15] 골든 상태! | AI확률: 0.4425 | 이평기울기: 하락
[2025-04-16] 골든 상태! | AI확률: 0.4488 | 이평기울기: 하락
[2025-04-17] 골든 상태! | AI확률: 0.4443 | 이평기울기: 상승
🚀 [2025-04-17] 진입! | 가격: 84,896 | 확률: 0.4443
[2025-04-18] 골든 상태! | AI확률: 0.4339 | 이평기울기: 상승
[2025-04-19] 골든 상태! | AI확률: 0.4092 | 이평기울기: 상승
[2025-04-20] 골든 상태! | AI확률: 0.3854 | 이평기울기: 상승
[2025-04-21] 골든 상태! | AI확률: 0.3741 | 이평기울기: 상승
[2025-04-22] 골든 상태! | AI확률: 0.3630 | 이평기울기: 상승
[2025-04-23] 골든 상태! | AI확률: 0.3476 | 이평기울기: 상승
[2025-04-24] 골든 상태! | AI확률: 0.3340 | 이평기울기: 상승
[2025-04-25] 골든 상태! | AI확률: 0.3230 | 이평기울기: 상승
[2025-04-26] 골든 상태! | AI확률: 0.3135 | 이평기울기: 상승
[2025-04-27] 골든 상태! | AI확률: 0.3266 | 이평기울기: 상승
[2025-04-28] 골든 상태! | AI확률: 0.3492 | 이평기울기: 상승
[2025-04-29] 골든 상태! | AI확률: 0.3649 | 이평기울기: 상승
[2025-04-30] 골든 상태! | AI확률: 0.3666 | 이평기울기: 상승
[2025-05-01] 골든 상태! | AI확률: 0.3779 | 이평기울기: 상승
[2025-05-02] 골든 상태! | AI확률: 0.3900 | 이평기울기: 상승
[2025-05-03] 골든

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = 'colab'

# 1. 시뮬레이션 및 데이터 기록용 변수
initial_balance = 10000000
balance_v4 = initial_balance
position_v4 = 0
buy_price = 0

# 그래프용 기록 리스트
dates = []
limitless_values = []
bh_values = []
buy_markers_x = []
buy_markers_y = []
sell_markers_x = []
sell_markers_y = []

# Buy & Hold 초기 설정
first_idx = prediction_days
bh_initial_price = df['Close'].iloc[first_idx]
bh_position = initial_balance / bh_initial_price

# 2. 백테스팅 및 일별 자산 기록 루프
for i in range(first_idx, len(df)):
    current_date = df.index[i]
    current_price = df['Close'].iloc[i]

    # AI 및 지표 데이터 (이전 성공 로직 그대로 사용)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수/매도 로직]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)
    is_ai_approve = prob > 0.38 # 24%를 만든 그 기준값!

    if position_v4 == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position_v4 = balance_v4 / buy_price
        balance_v4 = 0
        buy_markers_x.append(current_date)
        buy_markers_y.append(current_price)

    elif position_v4 > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance_v4 = position_v4 * current_price
            position_v4 = 0
            sell_markers_x.append(current_date)
            sell_markers_y.append(current_price)

    # 매일매일의 가치 기록
    dates.append(current_date)
    limitless_values.append(balance_v4 + (position_v4 * current_price))
    bh_values.append(bh_position * current_price)

# 3. 통합 차트 생성
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Limitless: BTC 시세 및 매매 타점', '누적 수익률 비교 (%)'),
                    row_heights=[0.6, 0.4])

# (1) 상단: 캔들스틱 + 이평선 + 매매마커
fig.add_trace(go.Candlestick(x=df.index[first_idx:], open=df['Open'].iloc[first_idx:], high=df['High'].iloc[first_idx:], low=df['Low'].iloc[first_idx:], close=df['Close'].iloc[first_idx:], name="BTC", opacity=0.4), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index[first_idx:], y=df['MA20'].iloc[first_idx:], name='MA20', line=dict(color='cyan', width=1)), row=1, col=1)

# 매수/매도 마커 추가
fig.add_trace(go.Scatter(x=buy_markers_x, y=buy_markers_y, mode='markers', marker=dict(symbol='triangle-up', size=12, color='lime'), name='매수(Entry)'), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_markers_x, y=sell_markers_y, mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'), name='매도(Exit)'), row=1, col=1)

# (2) 하단: 수익률 곡선 (%)
limitless_returns = [(v/initial_balance - 1)*100 for v in limitless_values]
bh_returns = [(v/initial_balance - 1)*100 for v in bh_values]

fig.add_trace(go.Scatter(x=dates, y=limitless_returns, name='Limitless 전략', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates, y=bh_returns, name='Buy & Hold', line=dict(color='gray', width=1, dash='dash')), row=2, col=1)

fig.update_layout(height=800, title='<b>Limitless v4.2</b> Performance Dashboard', template='plotly_dark', xaxis_rangeslider_visible=False)
fig.show()

## 다중 피처 학습(Multi-Feature)

In [ ]:
# 1. 추가 데이터 계산
df['RSI'] = ta.rsi(df['Close'], length=14)
df['Vol_Change'] = df['Volume'].pct_change() # 거래량 변화율 추가

# 결측치 제거 및 필요한 컬럼만 추출
features = df[['Close', 'RSI', 'Volume']].dropna()

# 2. 데이터 정규화 (각 피처별로 따로 스케일링)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# 3. 학습 데이터셋 생성 (입력 차원이 1에서 3으로 변경됨)
prediction_days = 60
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_features)):
    # x-60일부터 x일까지의 [Close, RSI, Volume] 3가지 데이터를 입력으로 사용
    x_train.append(scaled_features[x-prediction_days:x, :])
    # 정답은 여전히 'Close' 가격의 다음 날 값 (첫 번째 컬럼)
    y_train.append(scaled_features[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
model = Sequential()

# input_shape=(60, 3) -> 60일치 데이터, 3개의 피처
model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid')) # 과제 로직인 확률 출력을 위해 sigmoid 사용

model.compile(optimizer='adam', loss='binary_crossentropy') # 분류 모델에 최적화된 손실함수

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


# 3. 모델 학습 실행
history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2, # 데이터의 20%를 검증용으로 사용
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 226ms/step - loss: 0.6792 - val_loss: 0.8066
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - loss: 0.6449 - val_loss: 0.7982
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.6359 - val_loss: 0.7230
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.6047 - val_loss: 0.6835
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.5911 - val_loss: 0.6101
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.5835 - val_loss: 0.5957
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.5606 - val_loss: 0.5954
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.5693 - val_loss: 0.5972
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.5650 - val_loss: 0.5974
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.5556 - val_loss: 0.5993
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.5695 - val_loss: 0.5964
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.5570 - val_

In [ ]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측: 확률값 가져오기 (0~1 사이)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0] # Sigmoid 결과값

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1] # 20일선 기울기 확인용

    # [과제 기반 3중 필터 + AI 확률]
    # 1) 가격이 20일선 위 2) 20일선이 우상향 3) 5일선 > 20일선
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)

    # AI 조건: 확률이 0.465를 넘는지 확인
    is_ai_approve = prob > 0.38

    if is_golden_status:
        print(f"[{df.index[i].date()}] 골든 상태! | AI확률: {prob:.4f} | 이평기울기: {'상승' if curr_ma20 > prev_ma20 else '하락'}")

    # 최종 매수 결정
    if position == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"🚀 [{df.index[i].date()}] 진입! | 가격: {buy_price:,.0f} | 확률: {prob:.4f}")

    # [매도 조건: 과제 로직 - 익절 15%, 손절 -2%, 추세이탈 ma20]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # A. 익절 15% / B. 손절 -2% / C. 가격이 20일선 하향 돌파
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance = position * current_price
            history.append(profit_loss)
            reason = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 매도 ({reason}) | 수익률: {profit_loss*100:.2f}%")
            position = 0

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-03-30] 골든 상태! | AI확률: 0.1674 | 이평기울기: 상승
[2025-04-14] 골든 상태! | AI확률: 0.1866 | 이평기울기: 하락
[2025-04-15] 골든 상태! | AI확률: 0.1843 | 이평기울기: 하락
[2025-04-16] 골든 상태! | AI확률: 0.1843 | 이평기울기: 하락
[2025-04-17] 골든 상태! | AI확률: 0.1924 | 이평기울기: 상승
[2025-04-18] 골든 상태! | AI확률: 0.1942 | 이평기울기: 상승
[2025-04-19] 골든 상태! | AI확률: 0.1984 | 이평기울기: 상승
[2025-04-20] 골든 상태! | AI확률: 0.2028 | 이평기울기: 상승
[2025-04-21] 골든 상태! | AI확률: 0.2235 | 이평기울기: 상승
[2025-04-22] 골든 상태! | AI확률: 0.2975 | 이평기울기: 상승
[2025-04-23] 골든 상태! | AI확률: 0.3535 | 이평기울기: 상승
[2025-04-24] 골든 상태! | AI확률: 0.3755 | 이평기울기: 상승
[2025-04-25] 골든 상태! | AI확률: 0.3858 | 이평기울기: 상승
🚀 [2025-04-25] 진입! | 가격: 94,720 | 확률: 0.3858
[2025-04-26] 골든 상태! | AI확률: 0.3964 | 이평기울기: 상승
[2025-04-27] 골든 상태! | AI확률: 0.3771 | 이평기울기: 상승
[2025-04-28] 골든 상태! | AI확률: 0.3825 | 이평기울기: 상승
[2025-04-29] 골든 상태! | AI확률: 0.3818 | 이평기울기: 상승
[2025-04-30] 골든 상태! | AI확률: 0.3791 | 이평기울기: 상승
[2025-05-01] 골든 상태! | AI확률: 0.4114 | 이평기울기: 상승
[2025-05-02] 골든 상태! | AI확률: 0.4402 | 이평기울기: 상승
[2025-05-03] 골든

In [ ]:
pio.renderers.default = 'colab'

# 1. 시뮬레이션 및 데이터 기록용 변수
initial_balance = 10000000
balance_v4 = initial_balance
position_v4 = 0
buy_price = 0

# 그래프용 기록 리스트
dates = []
limitless_values = []
bh_values = []
buy_markers_x = []
buy_markers_y = []
sell_markers_x = []
sell_markers_y = []

# Buy & Hold 초기 설정
first_idx = prediction_days
bh_initial_price = df['Close'].iloc[first_idx]
bh_position = initial_balance / bh_initial_price

# 2. 백테스팅 및 일별 자산 기록 루프
for i in range(first_idx, len(df)):
    current_date = df.index[i]
    current_price = df['Close'].iloc[i]

    # AI 및 지표 데이터 (이전 성공 로직 그대로 사용)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수/매도 로직]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)
    is_ai_approve = prob > 0.38 # 24%를 만든 그 기준값!

    if position_v4 == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position_v4 = balance_v4 / buy_price
        balance_v4 = 0
        buy_markers_x.append(current_date)
        buy_markers_y.append(current_price)

    elif position_v4 > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance_v4 = position_v4 * current_price
            position_v4 = 0
            sell_markers_x.append(current_date)
            sell_markers_y.append(current_price)

    # 매일매일의 가치 기록
    dates.append(current_date)
    limitless_values.append(balance_v4 + (position_v4 * current_price))
    bh_values.append(bh_position * current_price)

# 3. 통합 차트 생성
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Limitless: BTC 시세 및 매매 타점', '누적 수익률 비교 (%)'),
                    row_heights=[0.6, 0.4])

# (1) 상단: 캔들스틱 + 이평선 + 매매마커
fig.add_trace(go.Candlestick(x=df.index[first_idx:], open=df['Open'].iloc[first_idx:], high=df['High'].iloc[first_idx:], low=df['Low'].iloc[first_idx:], close=df['Close'].iloc[first_idx:], name="BTC", opacity=0.4), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index[first_idx:], y=df['MA20'].iloc[first_idx:], name='MA20', line=dict(color='cyan', width=1)), row=1, col=1)

# 매수/매도 마커 추가
fig.add_trace(go.Scatter(x=buy_markers_x, y=buy_markers_y, mode='markers', marker=dict(symbol='triangle-up', size=12, color='lime'), name='매수(Entry)'), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_markers_x, y=sell_markers_y, mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'), name='매도(Exit)'), row=1, col=1)

# (2) 하단: 수익률 곡선 (%)
limitless_returns = [(v/initial_balance - 1)*100 for v in limitless_values]
bh_returns = [(v/initial_balance - 1)*100 for v in bh_values]

fig.add_trace(go.Scatter(x=dates, y=limitless_returns, name='Limitless 전략', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates, y=bh_returns, name='Buy & Hold', line=dict(color='gray', width=1, dash='dash')), row=2, col=1)

fig.update_layout(height=800, title='<b>Limitless v4.2</b> Performance Dashboard', template='plotly_dark', xaxis_rangeslider_visible=False)
fig.show()